In [1]:
import os
import cv2
import torch
import clip
from PIL import Image
from tqdm import tqdm
import shutil

# ===== CONFIGURATION =====
SOURCE_DIR = r"C:\Users\crazycyt\Desktop\1470\dataset2"
DEST_DIR = "filtered_dataset2"
if os.path.exists(DEST_DIR) and os.path.isdir(DEST_DIR):
    shutil.rmtree(DEST_DIR)
os.makedirs(DEST_DIR, exist_ok=True)

BLUR_THRESHOLD = 150
SCENE_SCORE_THRESHOLD = 27.0  # Adjust based on output
BATCH_SIZE = 32

# Use CLIP for "environmental richness"
SCENE_TEXTS = [
    # Muddy / Earthy / Woody
    "a muddy swamp with stagnant water and broken trees",
    "a wet forest path lined with mossy rocks",
    "a shadowy jungle trail covered in thick vines",
    "a rain-soaked battlefield with churned earth",
    "a tree-rooted cave deep in the forest",

    # Grassy / Floral / Sweet / Clean
    "a grassy hill at golden hour with wildflowers",
    "a highland valley filled with purple heather",
    "a clean mountain stream surrounded by flowers",
    "a sunlit forest clearing with sweet-scented blossoms",
    "a peaceful garden with colorful petals and chirping birds",

    # Rotten / Musty / Damp / Dusty
    "a flooded cellar with rotting barrels",
    "a moldy basement full of broken furniture",
    "a dusty crypt with cobwebs and dry bones",
    "a damp, mossy sewer tunnel with stagnant air",
    "a musty wooden attic in a ruined mansion",

    # Bloody / Burnt / Smoky / Sulfuric
    "a blood-soaked stone floor in a torture chamber",
    "a battlefield with burnt corpses and rising smoke",
    "a volcanic cave glowing with lava and sulfur fumes",
    "a forest on fire with trees crackling and smoke rising",
    "a scorched courtyard littered with burnt debris",

    # Chemical / Oily / Metallic
    "a dark industrial lab with glowing green vials",
    "an oily workshop with rusted machinery and tools",
    "a metallic ship interior with flickering lights",
    "a steampunk engine room leaking smoke and oil",
    "a broken reactor room with chemical leaks on the floor",

    # Arcane / Magical
    "an arcane observatory filled with glowing sigils",
    "a wizard’s study with floating runes and blue flames",
    "a cursed circle of stones crackling with magic energy",
    "a magical forest glowing with bioluminescent plants",
    "a ritual chamber with arcane symbols and flickering candles",

    # Cold / Snowy / Clean
    "a snowy mountain pass with icy wind and frozen trees",
    "a frost-covered village under soft moonlight",
    "a clean ice cave with sparkling crystals",
    "a blizzard-swept path through pine trees",
    "a frozen battlefield with blood stains in the snow",

    # Dry / Dusty / Desert-like
    "a dry desert ruin filled with sand and bones",
    "a sun-baked canyon with cracked earth",
    "an abandoned town in a windstorm of dust",
    "a dusty shrine with broken pillars and faded banners",

    # Swampy / Foul
    "a dark marsh with bubbling mud and dying trees",
    "a haunted bog with fog and glowing eyes in the dark",
    "a reeking cesspit filled with bones and slime",
    "a waterlogged ruin in a stinking swamp",

    # Urban Decay / Sewers / Waste
    "a collapsing tunnel filled with sewage and rust",
    "an underground canal choked with debris",
    "a toxic waste zone with barrels and glowing slime",
    "a post-apocalyptic alley with blood and chemicals",

    # Bonus atmospheric combos
    "a foggy castle hallway lit by torches",
    "a cursed battlefield with green mist and broken armor",
    "a magical forest tainted by rot and corruption",
    "a rain-drenched medieval village after a raid",
    "a shipwreck buried in sand and coral",
    "a battlefield at dawn with smoke drifting in the air"
]


# ===== FUNCTIONS =====
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
with torch.no_grad():
    text_tokens = clip.tokenize(SCENE_TEXTS).to(device)
    text_features = model.encode_text(text_tokens)
    text_features /= text_features.norm(dim=-1, keepdim=True)


def is_blurry(image_path, threshold=BLUR_THRESHOLD):
    try:
        img = cv2.imread(image_path)
        if img is None:
            return True
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        variance = cv2.Laplacian(gray, cv2.CV_64F).var()
        return variance < threshold
    except:
        return True


def clip_scene_score(image_path):
    try:
        image = preprocess(Image.open(image_path).convert("RGB")).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)
            image_features /= image_features.norm(dim=-1, keepdim=True)
            similarity = (100.0 * image_features @ text_features.T).squeeze().tolist()
        return max(similarity)  # Highest scene match
    except:
        return 0.0


# ===== MAIN LOOP =====
all_images = [os.path.join(SOURCE_DIR, f) for f in os.listdir(SOURCE_DIR)
              if f.lower().endswith(('.png', '.jpg', '.jpeg','.webp'))]

print(f"Total images: {len(all_images)}")

kept_count = 0
for path in tqdm(all_images, desc="Filtering images"):
    if is_blurry(path):
        continue
    score = clip_scene_score(path)
    if score >= SCENE_SCORE_THRESHOLD:
        shutil.copy(path, os.path.join(DEST_DIR, os.path.basename(path)))
        kept_count += 1

print(f"✅ Done! Kept {kept_count} high-quality, environment-rich images.")


Total images: 22704


Filtering images: 100%|██████████████████████████████████████████████████████████| 22704/22704 [59:32<00:00,  6.35it/s]

✅ Done! Kept 1289 high-quality, environment-rich images.
